#### Import des bibliothéques:

In [41]:
import numpy as np
from mip import Model, xsum, maximize

### 1. Fonction de base : PMR (Pairwise Maximum Regret)

- Calcule le regret maximal par paires (PMR) entre deux alternatives x et y.
    
    Args :
    - x (array): Première alternative.
    - y (array): Deuxième alternative.
    - m (Model): Modèle MIP pour optimiser les poids.
    
    Returns :
    - float : Valeur du regret maximal.

In [42]:
def PMR(x, y, m):
    
    m_crit = len(x)  # Nombre de critères pour les alternatives (taille de x et y)
    
    # Créer les variables de décision w_i pour chaque critère (poids des critères)
    w = [m.add_var(name=f"w[{i}]", lb=0) for i in range(m_crit)]
    
    # Ajouter une contrainte pour que la somme des poids soit égale à 1 (normalisation)
    m += xsum(w[i] for i in range(m_crit)) == 1
    
    # Définir l'objectif à maximiser : la différence pondérée entre y et x
    m.objective = maximize(xsum(w[i] * (y[i] - x[i]) for i in range(m_crit)))
    
    # Optimiser le modèle pour trouver la configuration de poids maximisant la différence
    m.optimize()
    
    # Retourner la valeur maximale de la différence obtenue (PMR) et les variables w
    return m.objective_value, w

### 2. Les fonctions MR et mMR:

##### Fonction MR:

- Calcule le regret maximal (MR) d'une alternative x par rapport à toutes les autres alternatives dans X.
    
    Args :
    - x (array): Alternative pour laquelle calculer le MR.
    - X (array): Ensemble des alternatives.
    - m (Model): Modèle MIP pour les calculs.
    
    Returns :
    - float : Valeur du regret maximal pour x.

In [43]:
def MR(x, X, m):

    # On doit s'assurer que l'on récupère seulement la première valeur de retour de PMR
    return max(PMR(x, y, m)[0] for y in X if not np.array_equal(x, y))  # Récupérer seulement le premier élément

##### Fonction mMR:

- Calcule le regret minimax pour trouver la meilleure alternative x* et la pire y*.
    
    Args :
    - X (array): Ensemble des alternatives.
    - m (Model): Modèle MIP pour les calculs.
    
    Returns :
    - float : Valeur du regret minimax.
    - array : Alternative x* qui minimise le regret.
    - array : Alternative y* qui maximise le regret par rapport à x*.

In [44]:
def mMR(X, m):

    # Trouver x* qui minimise le MR parmi toutes les alternatives de X
    x_star = min(X, key=lambda x: MR(x, X, m))
    
    # Trouver y* qui maximise le PMR par rapport à x*
    y_star = max(X, key=lambda y: PMR(x_star, y, m)[0])  # Assurez-vous d'appeler PMR correctement
    
    # Calculer le MR de x* pour obtenir le mMR
    max_regret = MR(x_star, X, m)  # max_regret doit être un float ici
    
    return max_regret, x_star, y_star  # Retourner max_regret comme un float

### Fonction M_Omega: 

In [45]:
# Fonction pour calculer le modèle de décision avec les poids
def M_omega(x, omega):
    return np.dot(x, omega)

### 3. Fonction pour générer les questions : getQuerySet

- Génère un ensemble de requêtes potentielles pour l'élucidation.

- Une requête est une question qui demande au décideur de comparer deux alternatives.
- Ici, les requêtes sont des comparaisons entre l'alternative actuelle x* et les autres alternatives.
    
    
    Args :
    - X (array): Ensemble des alternatives.
    - x_star (array): L'alternative actuellement optimale.
    - k (int): Nombre maximal de requêtes à générer.
    
    Returns :
    - list : Liste de tuples (x, y) représentant les requêtes possibles.

In [46]:
def getQuerySet(X, x_star, k):

    # Créer une liste de requêtes de la forme (x*, y) pour chaque alternative y dans X
    queries = [(x_star, y) for y in X if not np.array_equal(x_star, y)]
    
    # Limiter le nombre de requêtes à k pour éviter une explosion combinatoire
    return queries[:k]

### 4. Fonction pour échantillonner les poids : sampleWeights 

- Échantillonne des poids à partir du polytope Omega pour les simulations.

- Chaque échantillon représente une configuration possible de préférences que le décideur pourrait avoir, ce qui permet de simuler les réponses du décideur à une requête.
    
    Args :
    - omega (dict): Dictionnaire représentant les contraintes de normalisation de Omega
    - nb_rollouts (int): Nombre de simulations à effectuer.
    
    Returns :
    - list : Liste de poids échantillonnés.

In [47]:
def sampleWeights(omega, nb_rollouts): ### modifier

    # Déduire la dimension à partir de la longueur des coefficients de Omega.
    dimension = len(omega['coefficients'])
    
    # Générer les échantillons de poids avec une distribution de Dirichlet.
    weights_samples = np.random.dirichlet(np.ones(dimension), size=nb_rollouts)
    
    # Vérifier que chaque échantillon respecte les contraintes de Omega.
    valid_samples = []
    for sample in weights_samples:
        # Vérifier si le vecteur de poids respecte les contraintes de normalisation.
        if np.isclose(np.sum(sample * omega['coefficients']), omega['rhs']):
            valid_samples.append(sample)
    
    return valid_samples

### 5. Fonction pour mettre à jour le polytope : Update

- Met à jour le modèle MIP avec une nouvelle contrainte en fonction de la réponse du décideur.
    
- La réponse du décideur permet de réduire l'ensemble des poids possibles, en ajoutant une contrainte au modèle qui reflète la préférence révélée.
    
    Args :
    - m (Model): Modèle MIP.
    - answer (array): Alternative préférée.
    - x (array), y (array): Alternatives comparées.
    
    Returns :
    - Model : Modèle MIP mis à jour.

In [48]:
def Update(m, answer, x, y, w):

    # Si le décideur préfère x à y, on ajoute la contrainte M_omega(x) >= M_omega(y)
    if np.array_equal(answer, x):
        new_constraint = xsum((x[i] - y[i]) * w[i] for i in range(len(x))) >= 0
    # Sinon, on ajoute la contrainte M_omega(y) >= M_omega(x)
    else:
        new_constraint = xsum((y[i] - x[i]) * w[i] for i in range(len(x))) >= 0
    
    # Ajouter la nouvelle contrainte au modèle.
    m += new_constraint
    return m

### 6. Fonction simulateRollout

- Simule un rollout de l'élicitation en utilisant un ensemble de poids omega_sample.
    
    Args :
    - X (array): Ensemble des alternatives possibles.
    - query (tuple): La première question de la forme (x, y).
    - m (Model): Modèle MIP initial représentant les contraintes sur les poids.
    - omega_sample (list): Un échantillon de poids représentant les préférences simulées du décideur.
    - epsilon (float): Tolérance pour le regret minimax, pour décider quand s'arrêter.
    
    Returns :
    - int: Le nombre de questions posées jusqu'à ce que le processus soit terminé.

In [54]:
def simulateRollout(X, query, m, omega_sample, epsilon):
    
    # Créer une copie du modèle pour ce rollout
    m_copy = m.copy()
    
    x, y = query
    total_queries = 0
    
    # Simuler le processus d'élicitation avec le jeu de poids omega_sample
    max_regret, x_star, y_star = mMR(X, m_copy)
    
    # Tant que le regret minimax est supérieur à epsilon, continuer à poser des questions.
    while max_regret >= epsilon:
        # Simuler la réponse du décideur en fonction de omega_sample pour la requête (x, y).
        if M_omega(x_star, omega_sample) >= M_omega(y_star, omega_sample):
            answer = x_star
        else:
            answer = y_star

        # Mettre à jour le modèle MIP avec la réponse simulée.
        _, w = PMR(x_star, y_star, m_copy)  # Récupérer les poids pour la comparaison
        m_copy = Update(m_copy, answer, x_star, y_star, w)
        
        # Recalculer le regret minimax après la mise à jour.
        max_regret, x_star, y_star = mMR(X, m_copy)
        
        # Incrémenter le compteur de questions posées.
        total_queries += 1
    
    return total_queries

### 7. Fonction pour poser la question et obtenir la réponse : getAnswer

- Simule la réponse du décideur en fonction de l'évaluation des alternatives à l'aide d'un échantillon de poids omega_sample.
    
    Args :
    - query (tuple): La question sous forme de (x, y), où x et y sont des alternatives.
    - omega_sample (list): Un vecteur de poids simulant les préférences du décideur.
    
    Returns :
    - array: L'alternative préférée entre x et y.

In [50]:
def getAnswer(query, omega_sample):

    x, y = query
    
    # Calculer M_omega pour les deux alternatives avec les poids omega_sample.
    value_x = M_omega(x, omega_sample)
    value_y = M_omega(y, omega_sample)
    
    # Retourner l'alternative qui a la plus grande valeur.
    if value_x >= value_y:
        return x
    else:
        return y

## Fonction principale : mcs_incremental_elicitation

- Implémente l'algorithme MCS Incremental Elicitation pour affiner les préférences du décideur.
    
- L'objectif est de trouver une alternative qui minimise le regret minimax en posant des questions au décideur, de manière à réduire progressivement l'incertitude sur ses préférences.
    
    Args :
    - X (array): Ensemble des alternatives possibles.
    - m (Model): Modèle MIP initial représentant les contraintes sur les poids.
    - epsilon (float): Tolérance pour le regret minimax, pour décider quand s'arrêter.
    - k (int): Nombre de requêtes potentielles à générer lors de chaque itération.
    - nb_rollouts (int): Nombre de simulations Monte Carlo à effectuer pour chaque requête.
    
    Returns :
    - array : L'alternative recommandée (x*).

In [51]:
def mcs_incremental_elicitation(X, m, omega, epsilon, k=5, nb_rollouts=10):

    #  Calculer le regret minimax initial et les alternatives associées.
    max_regret, x_star, y_star = mMR(X, m)
    
    # Vérification des valeurs retournées par mMR
    print(f"Valeurs retournées par mMR: max_regret={max_regret}, x_star={x_star}, y_star={y_star}")
    
    # Boucle jusqu'à ce que le regret minimax soit inférieur à la tolérance epsilon.
    while max_regret >= epsilon:
        print(f"Regret actuel: {max_regret}")

        # Obtenir les requêtes potentielles (comparaisons entre x* et d'autres alternatives).
        queries = getQuerySet(X, x_star, k)
        best_score = float('inf')  # Initialiser le meilleur score à l'infini.
        best_query = None  # Initialiser la meilleure requête.
        
        # Échantillonner des poids à partir du polytope Ω.
        omega_samples = sampleWeights(omega, nb_rollouts)
        
        # Évaluer chaque requête en simulant des rollouts.
        for query in queries:
            total_queries = 0
            
            # Pour chaque échantillon de poids, simuler le processus de questionnement.
            for omega_sample in omega_samples:
                total_queries += simulateRollout(X, query, m, omega_sample,epsilon)
            
            # Calculer le score de la requête (nombre moyen de questions nécessaires).
            score_query = total_queries / nb_rollouts
            
            # Mettre à jour la meilleure requête si le score est meilleur.
            if score_query < best_score:
                best_score = score_query
                best_query = query
        
        # Poser la meilleure question au décideur et obtenir sa réponse.
        answer = getAnswer(best_query)
        
        # Mettre à jour le modèle MIP avec la nouvelle contrainte basée sur la réponse.
        _, w = PMR(best_query[0], best_query[1], m)  # Récupérer les poids
        m = Update(m, answer, best_query[0], best_query[1], w)
        
        # Recalculer le regret minimax.
        max_regret, x_star, y_star = mMR(X, m)
    
    # Retourner l'alternative optimale trouvée (x*).
    print(f"Regret minimax final: {max_regret}")
    return x_star
        


- L'algorithme MCS Incremental Elicitation cherche à trouver la meilleure alternative tout en posant un nombre minimal de questions au décideur.
- Il combine des techniques de Monte Carlo (pour simuler des scénarios possibles) avec la programmation linéaire (pour modéliser les préférences et les contraintes).
- À chaque question posée, l'incertitude sur les préférences du décideur est réduite, ce qui permet de converger vers une solution optimale.

#### Exemple d'utilisation de l'algorithme : exemple_elicitation

In [52]:
def exemple_elicitation():
    
    # Ensemble des alternatives (chaque alternative a deux critères).
    X = np.array([[0.5, 0.2], [0.7, 0.1], [0.6, 0.3], [0.4, 0.4]])
    
    # Initialiser un modèle MIP pour représenter les contraintes sur les poids.
    m = Model()

    omega = {'coefficients': [1, 1], 'rhs': 1}
    
    # Tolérance pour le regret minimax (plus elle est petite, plus l'algorithme est précis).
    epsilon = 0.1
    
    # Appeler la fonction principale pour exécuter l'élicitation.
    solution = mcs_incremental_elicitation(X, m, omega, epsilon)
    
    # Afficher la solution finale recommandée.
    print(f"\nSolution finale recommandée: {solution}") 

In [53]:
exemple_elicitation()

Starting solution of the Linear programming problem using Dual Simplex

Coin0506I Presolve 0 (-1) rows, 0 (-2) columns and 0 (-2) elements
Clp0000I Optimal - objective value 0.2
Coin0511I After Postsolve, objective 0.2, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 0.2 - 0 iterations time 0.002, Presolve 0.00
Starting solution of the Linear programming problem using Dual Simplex

Clp0006I 0  Obj -0 Primal inf 0.999999 (1) Dual inf 2e+10 (2)
Clp0000I Optimal - objective value 0.1
Starting solution of the Linear programming problem using Dual Simplex

Clp0006I 0  Obj -0 Primal inf 0.999999 (1) Dual inf 2e+10 (2)
Clp0000I Optimal - objective value 0.2
Starting solution of the Linear programming problem using Dual Simplex

Clp0006I 0  Obj -0 Primal inf 0.999999 (1) Dual inf 2e+10 (2)
Clp0000I Optimal - objective value 0.1
Starting solution of the Linear programming problem using Dual Simplex

Clp0006I 0  Obj -0 Primal inf 0.999999 (1) Dual inf 2e+10 (2)
Clp0000I Opt

Starting solution of the Linear programming problem using Dual Simplex

Clp0006I 0  Obj -0 Primal inf 0.999999 (1) Dual inf 2e+10 (2)
Clp0000I Optimal - objective value 0.1
Starting solution of the Linear programming problem using Dual Simplex

Clp0006I 0  Obj -0 Primal inf 0.999999 (1) Dual inf 2e+10 (2)
Clp0000I Optimal - objective value 0.1
Starting solution of the Linear programming problem using Dual Simplex

Clp0006I 0  Obj -0 Primal inf 0.999999 (1) Dual inf 2e+10 (2)
Clp0000I Optimal - objective value 0.2
Starting solution of the Linear programming problem using Dual Simplex

Clp0006I 0  Obj -0 Primal inf 0.999999 (1) Dual inf 2e+10 (2)
Clp0000I Optimal - objective value 0.1
Starting solution of the Linear programming problem using Dual Simplex

Clp0006I 0  Obj -0 Primal inf 0.999999 (1) Dual inf 2e+10 (2)
Clp0000I Optimal - objective value 0.2
Starting solution of the Linear programming problem using Dual Simplex

Clp0006I 0  Obj -0 Primal inf 0.999999 (1) Dual inf 2e+10 (2)
C

KeyboardInterrupt: 